In [1]:
from grpc.beta import implementations
import tensorflow as tf
import numpy
import pandas
from data_processing import test_data_with_label

from tensorflow.core.framework import types_pb2
from tensorflow.python.platform import flags
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2

In [2]:
HOST = "localhost"
PORT = 666
MNAME = "model"
MSIGNNAME = "predict"

In [3]:
# Getting test data from script
testing_images = test_data_with_label()
tst_img_data = numpy.array([i[0] for i in testing_images])
tst_lbl_data = numpy.array([i[1] for i in testing_images])

100%|██████████| 7/7 [00:00<00:00, 114.05it/s]


In [5]:
print(tst_img_data)
print(tst_img_data.size)

[[ 11.   5.  28. ... 203. 188. 167.]
 [177.  81.  60. ... 190. 166.  32.]
 [  0.   0.   0. ...  63. 102.  41.]
 ...
 [  0.   0.   0. ... 173. 127.  90.]
 [  8.   6.   8. ... 194. 187. 184.]
 [  4.   3.   4. ...   4.   2.  42.]]
5488


In [7]:
# Prepare request
request = predict_pb2.PredictRequest()
request.model_spec.signature_name = MSIGNNAME
request.model_spec.name = MNAME
request.inputs['input'].dtype = types_pb2.DT_INT32
request.inputs['input'].CopyFrom(
    tf.contrib.util.make_tensor_proto(tst_img_data[-1].astype(dtype=numpy.float32).reshape(1, tst_img_data.shape[1])))
request.inputs['prob'].CopyFrom(tf.contrib.util.make_tensor_proto(0.8))
request.output_filter.append('output')

In [8]:
# Send request
channel = implementations.insecure_channel(HOST, int(PORT))
stub = prediction_service_pb2.beta_create_PredictionService_stub(channel)
prediction = stub.Predict(request, 5.0)  # 5 secs timeout
floats = prediction.outputs['output'].float_val
pred_arr = numpy.array(floats)

In [9]:
print(pred_arr)

[0.32924139 0.22470589 0.44605276]
